# Download data 


Downloading the ECG 2020 datasets and copy to google drive

In [ ]:
# !wget -r -N -c -np https://physionet.org/files/ptb-xl/1.0.1/ 

In [ ]:
# %cp -r -np /content/physionet.org/files/ptb-xl /content/gdrive/My\ Drive/ECG2020new

In [ ]:
# #%cd /content/physionet.org/files/ptb-xl/
# %cd 20000/

In [ ]:
# %ls

In [ ]:
# %pwd

In [ ]:
#%cd physionet.org/files/ptb-xl/1.0.1/

Load GPU

In [ ]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Link to Google Drive Import library

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


Install library

In [ ]:
!pip install wfdb

     |████████████████████████████████| 122kB 15.4MB/s 


Import library

In [2]:
import numpy as np
import pandas as pd
import wfdb
import matplotlib.pyplot as plt
import ast
import os
import tensorflow as tf
from keras.utils import to_categorical

#import tensorflow_datasets as tfds

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# Get the label from csv, set up the basic raw data, get the features

In [3]:
#get path
path='/Users/xianghuihuang/Desktop/WFDB/'
# path = '/content/gdrive/.shortcut-targets-by-id/1ca8px2VUzY6uBCgo13kYoW0_JuynXfG7/1.0.1/'
tf_save_path = '/content/gdrive/My Drive/tfrecord/'

In [47]:
def get_label_info(path):
  def add_path(str2):
      return path + "HR"+str2.split("_")[0]
  def aggregate_diagnostic(y_dic):
      tmp = []
      for key in y_dic.keys():
          if key in agg_df.index:
              tmp.append(agg_df.loc[key].diagnostic_class)
      if len(tmp) == 0:
        return "NL"
      return list(set(tmp))[0]
  Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
  Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

      # Load scp_statements.csv for diagnostic aggregation
  agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
  agg_df = agg_df[agg_df.diagnostic == 1]

      # Apply diagnostic superclass
      #target label of the classification
  Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)
  Y['new_filename'] = Y.filename_hr.map(add_path)
  return Y[["new_filename","diagnostic_superclass","age","sex"]]

In [48]:
y = get_label_info(path)
y.head()

,new_filename,diagnostic_superclass,age,sex
ecg_id,,,,
1,/Users/xianghuihuang/Desktop/WFDB/HR00001,NORM,56.0,1
2,/Users/xianghuihuang/Desktop/WFDB/HR00002,NORM,19.0,0
3,/Users/xianghuihuang/Desktop/WFDB/HR00003,NORM,37.0,1
4,/Users/xianghuihuang/Desktop/WFDB/HR00004,NORM,24.0,0
5,/Users/xianghuihuang/Desktop/WFDB/HR00005,NORM,19.0,1
6,/Users/xianghuihuang/Desktop/WFDB/HR00006,NORM,18.0,1
7,/Users/xianghuihuang/Desktop/WFDB/HR00007,NORM,54.0,0
8,/Users/xianghuihuang/Desktop/WFDB/HR00008,MI,48.0,0
9,/Users/xianghuihuang/Desktop/WFDB/HR00009,NORM,55.0,0


In [49]:
y = y[y.diagnostic_superclass !="DELETE"]
y.diagnostic_superclass.value_counts()

NORM    9083
STTC    5033
MI      4122
CD      2654
HYP      538
NL       407
Name: diagnostic_superclass, dtype: int64

In [50]:
y = y.dropna(how='any',axis=0) 
y.diagnostic_superclass.value_counts()

NORM    9077
STTC    4999
MI      4100
CD      2631
HYP      538
NL       403
Name: diagnostic_superclass, dtype: int64

Convert the label into numercial data

In [51]:
numeric_label = {"diagnostic_superclass":{"NORM":0,"CD":1,"MI":2,"STTC":3,"HYP":4,"NL":5}}

In [52]:
y = y.replace(numeric_label)
y.head()

,new_filename,diagnostic_superclass,age,sex
ecg_id,,,,
1,/Users/xianghuihuang/Desktop/WFDB/HR00001,0,56.0,1
2,/Users/xianghuihuang/Desktop/WFDB/HR00002,0,19.0,0
3,/Users/xianghuihuang/Desktop/WFDB/HR00003,0,37.0,1
4,/Users/xianghuihuang/Desktop/WFDB/HR00004,0,24.0,0
5,/Users/xianghuihuang/Desktop/WFDB/HR00005,0,19.0,1
6,/Users/xianghuihuang/Desktop/WFDB/HR00006,0,18.0,1
7,/Users/xianghuihuang/Desktop/WFDB/HR00007,0,54.0,0
8,/Users/xianghuihuang/Desktop/WFDB/HR00008,2,48.0,0
9,/Users/xianghuihuang/Desktop/WFDB/HR00009,0,55.0,0


In [71]:
y.new_filename.isnull().value_counts()

False    21748
Name: new_filename, dtype: int64

In [72]:
y.diagnostic_superclass.isnull().value_counts()

False    21748
Name: diagnostic_superclass, dtype: int64

Resampling the data label

In [53]:
#up sampling the miniority class of train dataset

df_0 = y[y.diagnostic_superclass==0]
df_1 = y[y.diagnostic_superclass==1]
df_2 = y[y.diagnostic_superclass==2]
df_3 = y[y.diagnostic_superclass==3]
df_4 = y[y.diagnostic_superclass==4]
df_0 = resample(df_0, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=123) # reproducible results
df_1 = resample(df_1, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=123) # reproducible results
df_2 = resample(df_2, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=123) # reproducible results
df_3 = resample(df_3, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=123) # reproducible results
df_4 = resample(df_4, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=123) # reproducible results                                
df_ = pd.concat([df_0,df_1,df_2,df_3,df_4],ignore_index=True)
df_.diagnostic_superclass.value_counts()

4    5000
3    5000
2    5000
1    5000
0    5000
Name: diagnostic_superclass, dtype: int64

In [54]:
#shuffle the dataframe and re_index the dataset
def shuffle_dataset(df):
  df = df.sample(frac=1)
  df = df.reset_index()
  return df
shuffle_dataset(df_).head()

,index,new_filename,diagnostic_superclass,age,sex
0,19801,/Users/xianghuihuang/Desktop/WFDB/HR05586,3,50.0,1
1,2845,/Users/xianghuihuang/Desktop/WFDB/HR19724,0,33.0,1
2,5316,/Users/xianghuihuang/Desktop/WFDB/HR10505,1,25.0,1
3,24556,/Users/xianghuihuang/Desktop/WFDB/HR04458,4,54.0,0
4,16636,/Users/xianghuihuang/Desktop/WFDB/HR14952,3,53.0,0
5,6629,/Users/xianghuihuang/Desktop/WFDB/HR21675,1,66.0,1
6,19655,/Users/xianghuihuang/Desktop/WFDB/HR02856,3,65.0,1
7,21025,/Users/xianghuihuang/Desktop/WFDB/HR07857,4,73.0,1
8,24322,/Users/xianghuihuang/Desktop/WFDB/HR06428,4,69.0,1
9,24808,/Users/xianghuihuang/Desktop/WFDB/HR01088,4,57.0,0


In [55]:
#split the train test dataset
train_df, test_df = train_test_split(df_, test_size = 0.2)
#split the train validation dataset
train_df, val_df = train_test_split(train_df, test_size = 0.2)

In [130]:
train_df = train_df.reset_index()
test_df = test_df.reset_index()
val_df = val_df.reset_index()

In [131]:
print("train dataset information:")
print(train_df.info())
print()
print("test dataset information:")
print(test_df.info())
print()
print("validation dataset information:")
print(val_df.info())

train dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 5 columns):
index                    16000 non-null int64
new_filename             16000 non-null object
diagnostic_superclass    16000 non-null int64
age                      16000 non-null float64
sex                      16000 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 625.1+ KB
None

test dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
index                    5000 non-null int64
new_filename             5000 non-null object
diagnostic_superclass    5000 non-null int64
age                      5000 non-null float64
sex                      5000 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 195.4+ KB
None

validation dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 5 colu

In [148]:
train_df.new_filename[0]

'/Users/xianghuihuang/Desktop/WFDB/HR01195'

# Read data into Tensorflow data pipeline

TensorFlow Encode and Decode base Function

In [133]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def save_tfrecords(data, label, sex, age, desfile):
    with tf.io.TFRecordWriter(desfile) as writer:
      features = tf.train.Features(
                feature = {
                    "data": _bytes_feature(tf.io.serialize_tensor(data)),
                    "label":_int64_feature(label),
                    "sex":_int64_feature(sex),
                    "age":_float_feature(age)
                }
            )
      example = tf.train.Example(features = features)
      serialized = example.SerializeToString()
      writer.write(serialized)

**here can load the raw data directly to memory, sampling_rate 100 means the 100hz dataset, physinet provide 100hz dataset for user easier to use but we use 500 hz dataset in our project here**

In [134]:
# here can load the raw data directly to memory, sampling_rate 100 means the 100hz dataset, physinet provide 100hz dataset for user easier to use
# but we use 500 hz dataset in our project here

# def load_raw_data(path):
#     print("start")
#     sampling_rate = 100
#     def aggregate_diagnostic(y_dic):
#       tmp = []
#       for key in y_dic.keys():
#           if key in agg_df.index:
#               tmp.append(agg_df.loc[key].diagnostic_class)
#       return list(set(tmp))
#     # load and convert annotation data
#     Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
#     Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

#     # Load scp_statements.csv for diagnostic aggregation
#     agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
#     agg_df = agg_df[agg_df.diagnostic == 1]

#     # Apply diagnostic superclass
#     #target label of the classification
#     Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)
#     print(len(Y.diagnostic_superclass))
#     #Y.reset_index()
#     # 100 HZ dataset
#     # if sampling_rate == 100:
#     #     data = [wfdb.rdrecord(path+f).p_signal.T for f in Y.filename_lr]
#     #     #print(data)
#     # else:
#     # 500 HZ array dataset
#     for i in range(1,len(Y)+1):
#       f = Y.filename_hr[i]
#       # print("file name:")
#       # print(f)
#       sex = Y.sex[i]
#       # print("sex:")
#       # print(sex)
#       age = Y.age[i]
#       # print("age:")
#       # print(age)
#       data =wfdb.rdrecord(path+f).p_signal.T
#       # print("data:"+str(i))
#       # print(data)
#       tf_path = os.path.join(tf_save_path,f+'.tfrecords')
#       # print("tf_path:")
#       # print(tf_path)
#       label = Y.diagnostic_superclass[i]
#       # print("label:")
#       # print(label)
#       save_tfrecords(data,label,sex,age,tf_path)
#     #data = [wfdb.rdrecord(path+f).p_signal.T for f in Y.filename_hr]
#     #data = np.array([signal for signal, meta in data])
#     print('finish')
#     return data, Y.diagnostic_superclass

In [214]:
#tensor flow function
def load_data(df,batch_size,i):
  #print(file_path)
  #df.reindex()
  end_index = (i+1)*batch_size
  if end_index > df.index[-1]:
    end_index = df.index[-1]
  start_index = i*batch_size
  #print('start:',start_index)
  #print('end:',end_index)
  path_list = df['new_filename'][start_index:end_index]
  
  #print(i)
  #age_tensor = tf.constant(df['age'][start_index: end_index],dtype="int8")
  #sex_tensor =tf.constant(df['sex'][start_index: end_index],dtype="int8")
  #label_one_hot = tf.one_hot(df['diagnostic_superclass'][start_index: end_index],5)
  #label_tensor = tf.constant(label_one_hot,dtype="float32")
  #file = file_path.numpy().decode('UTF-8')
  #print(file)
  data_list = []
  path_list=path_list.reset_index()
  for j in range(len(path_list)):
    #print(j)
    #print(len(path_list))
    #print(path_list.index[j])
    #print(path_list[1])
    file_path = path_list.new_filename[j]
    
    data = wfdb.rdrecord(file_path).p_signal.T.astype("float16")
    if data is None:
        print(file_path,"error!!")
    data_list.append(data)
  data_array = np.array(data_list)
  #dataset = tf.data.Dataset.from_tensor_slices((data_array,age_tensor,sex_tensor,label_tensor))
  #set tensorflow feature
  label_tensor = tf.constant(train_df['diagnostic_superclass'][start_index: end_index],5)
  dataset = tf.data.Dataset.from_tensor_slices((data_array, label_tensor))
  return dataset

def one_hot_encode(signal,label):
    if label == 0:
      label = np.array([1.0,.0,.0,.0,.0]).astype("float16")
    elif label == 1:
      label = np.array([.0,1.0,.0,.0,.0]).astype("float16")
    elif label ==3:
      label = np.array([.0,.0,1.0,.0,.0]).astype("float16")
    elif label ==4:
      label = np.array([.0,.0,.0,1.0,.0]).astype("float16")
    else: 
      label = np.array([.0,.0,.0,.0,1.0]).astype("float16")
    return signal , label

# data preprocessing 

In [215]:
import pywt

def denoise(sig_data):
    # wavedec
    coeffs = pywt.wavedec(data=sig_data, wavelet='db8', level=9)
    cA9, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs
    #print('here')
    # threshold
    threshold = (np.median(np.abs(cD1)) / 0.6745) * (np.sqrt(2 * np.log(len(cD1))))
    cD1.fill(0)
    cD2.fill(0)
    for i in range(1, len(coeffs) - 2):
       coeffs[i] = pywt.threshold(coeffs[i], threshold)

    rdata = pywt.waverec(coeffs=coeffs, wavelet='db8')
    #print(rdata)
    return rdata

def denoise_for_tf_data(data,label):
  # Generating numpy array 
  data_array = np.array(data)
  #print(data_array)
  for i, sig_array in enumerate(data_array):     
    #print(type(sig_array))
    # plt.figure()
    # plt.plot(sig_array[0:1024]) 
    # plt.show()
    #print(type(sig_array))
    sig_length = len(sig_array)
    sig_array = denoise(sig_array)
    #print(sig_array)
    #plt.plot(sig_array[0:1024])
    #plt.show()
    data_array[i] = sig_array[0:sig_length]
  #print(data_array.shape)
  data = tf.convert_to_tensor(data_array)
  #print(data.numpy())
  return data,label

In [241]:
def get_dataset(df,batch_size, i):
    dataset = load_data(df, batch_size,i)
    ds_one_hot = dataset.map(one_hot_encode)
    ds_denoise = ds_one_hot.map(lambda x,y: 
                              tuple(tf.py_function(denoise_for_tf_data,[x,y],[tf.float64,tf.float16])))
    ds_batch = ds_one_hot.batch(128)
    return ds_batch

In [242]:
def show(data, label):
  for sig_data in data:
    plt.figure()
    plt.title(label)
    plt.plot(sig_data[0:1024])
    plt.show()

In [243]:
# test function
load_data(train_df,512, 0)

<TensorSliceDataset shapes: ((12, 5000), ()), types: (tf.float16, tf.int16)>

In [244]:
# test function
ds_test0 = load_data(train_df,512, 1)
def show_dataset(dataset):
  iterator = iter(dataset)
  raw_example = next(iterator)
  parsed_data = raw_example[0].numpy()
  label = raw_example[1].numpy()
  print('batch_data:',parsed_data, 'shape:', parsed_data.shape, '\n batcah_labels:',label, 'shape:', label.shape)

show_dataset(ds_test0)

batch_data: [[-0.01  -0.01  -0.01  ... -0.045 -0.045 -0.045]
 [-0.08  -0.08  -0.08  ...  0.15   0.15   0.15 ]
 [-0.07  -0.07  -0.07  ...  0.195  0.195  0.195]
 ...
 [ 0.07   0.07   0.07  ... -0.01  -0.01  -0.01 ]
 [ 0.025  0.025  0.025 ... -0.17  -0.17  -0.17 ]
 [-0.075 -0.075 -0.075 ... -0.14  -0.14  -0.14 ]] shape: (12, 5000) 
 batcah_labels: 0 shape: ()


# Modeling

In [245]:
# %tensorflow_version 2.x
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [246]:
def buildModel():

    newModel = tf.keras.models.Sequential([
                                           
        tf.keras.layers.InputLayer(input_shape=(12,5000)),

        tf.keras.layers.Conv1D(filters=4, kernel_size=21, strides=1, padding='SAME', activation='relu'),
       
        tf.keras.layers.AvgPool1D(pool_size=1, strides=4, padding='SAME'),
        
        tf.keras.layers.Conv1D(filters=16, kernel_size=23, strides=1, padding='SAME', activation='relu'),
        
        tf.keras.layers.AvgPool1D(pool_size=1, strides=4, padding='SAME'),
       
        tf.keras.layers.Conv1D(filters=32, kernel_size=25, strides=1, padding='SAME', activation='elu'),
       
        tf.keras.layers.AvgPool1D(pool_size=1, strides=8, padding='SAME'),
       
        tf.keras.layers.Conv1D(filters=64, kernel_size=27, strides=1, padding='SAME', activation='relu'),
       
        tf.keras.layers.Flatten(),
       
        tf.keras.layers.Dense(128, activation='relu'),
       
        tf.keras.layers.Dropout(rate=0.2),
       
        tf.keras.layers.Dense(5, activation='softmax')
    ])

    # newModel.compile(optimizer = 'adam' , 
    #               loss = "categorical_crossentropy", metrics=["accuracy"])

    return newModel

In [247]:
# with tpu_strategy.scope():
model_test = buildModel()
model_test.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_24 (Conv1D)           (None, 12, 4)             420004    
_________________________________________________________________
average_pooling1d_18 (Averag (None, 3, 4)              0         
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 3, 16)             1488      
_________________________________________________________________
average_pooling1d_19 (Averag (None, 1, 16)             0         
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 1, 32)             12832     
_________________________________________________________________
average_pooling1d_20 (Averag (None, 1, 32)             0         
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 1, 64)            

In [248]:
import datetime
from keras.optimizers import RMSprop, Adam
  #tbcb = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,  write_graph=True, write_images=True)
%load_ext tensorboard

log_dir = "logs/fit_Feb26/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
model_test.compile(optimizer = 'adam', 
                  loss = "categorical_crossentropy", metrics=["accuracy"])

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Training

In [249]:
def train_model(train_df,batch_size , epochs):
  for i in range(int(len(train_df)/batch_size)+1):
    print('round:', i)  
    ds_batch = get_dataset(train_df,batch_size,i)
    train_prefetch = ds_batch.prefetch(tf.data.experimental.AUTOTUNE)
    train_cache = train_prefetch.cache()
    model_test.fit(train_cache, epochs=epochs,callbacks = [tensorboard_callback])

In [250]:
train_model(train_df, 2048, 5)

round: 0
Epoch 1/5
16/16 [==============================] - 4s 234ms/step - loss: 1.6093 - accuracy: 0.2078
Epoch 2/5
16/16 [==============================] - 4s 222ms/step - loss: 1.5972 - accuracy: 0.2332
Epoch 3/5
16/16 [==============================] - 4s 220ms/step - loss: 1.5004 - accuracy: 0.3449
Epoch 4/5
16/16 [==============================] - 4s 220ms/step - loss: 1.3268 - accuracy: 0.4130
Epoch 5/5
16/16 [==============================] - 4s 236ms/step - loss: 1.1048 - accuracy: 0.5127
round: 1
Epoch 1/5
16/16 [==============================] - 4s 223ms/step - loss: 1.7115 - accuracy: 0.3257
Epoch 2/5
16/16 [==============================] - 4s 222ms/step - loss: 1.3215 - accuracy: 0.4487
Epoch 3/5
16/16 [==============================] - 4s 225ms/step - loss: 1.1333 - accuracy: 0.5356
Epoch 4/5
16/16 [==============================] - 4s 226ms/step - loss: 0.9219 - accuracy: 0.6543
Epoch 5/5
16/16 [==============================] - 4s 239ms/step - loss: 0.7206 - accuracy: